<a href="https://colab.research.google.com/github/DionesGouvea/Tera/blob/main/keras_F1_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import Callback
from keras.optimizers import Adam,Adamax,Nadam
from tensorflow.keras import regularizers

# define o callback para salvar o modelo com a melhor métrica F1
class F1ScoreModelCheckpoint(Callback):
    def __init__(self, filepath, monitor='val_f1_score', mode='max', verbose=0, save_best_only=True, save_weights_only=False):
        super(F1ScoreModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.verbose = verbose
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.best_score = -np.inf
 
    def on_epoch_end(self, epoch, logs=None):
        score = logs[self.monitor]
        if self.mode == 'max':
            if score > self.best_score:
                if self.verbose > 0:
                    print(f"\nEpoch {epoch+1}: {self.monitor} improved from {self.best_score:.4f} to {score:.4f}")
                self.best_score = score
                if self.save_weights_only:
                    self.model.save_weights(self.filepath, overwrite=True)
                else:
                    self.model.save(self.filepath, overwrite=True)
            elif self.verbose > 0:
                print(f"\nEpoch {epoch+1}: {self.monitor} did not improve from {self.best_score:.4f}")
        else:
            if score < self.best_score:
                if self.verbose > 0:
                    print(f"\nEpoch {epoch+1}: {self.monitor} improved from {self.best_score:.4f} to {score:.4f}")
                self.best_score = score
                if self.save_weights_only:
                    self.model.save_weights(self.filepath, overwrite=True)
                else:
                    self.model.save(self.filepath, overwrite=True)
            elif self.verbose > 0:
                print(f"\nEpoch {epoch+1}: {self.monitor} did not improve from {self.best_score:.4f}")




# define as camadas da rede com regularização L2
input_layer = Input(shape=(X.shape[1],))
hidden_layer_1 = Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01))(input_layer)
hidden_layer_2 = Dense(80, activation="relu", kernel_regularizer=regularizers.l2(0.01))(hidden_layer_1)
hidden_layer_3 = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(hidden_layer_2)
hidden_layer_4 = Dense(175, activation="relu", kernel_regularizer=regularizers.l2(0.01))(hidden_layer_3)
#hidden_layer_5 = Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01))(hidden_layer_4)
#hidden_layer_6 = Dense(1024, activation="relu", kernel_regularizer=regularizers.l2(0.01))(hidden_layer_4)
# concatena as camadas
concat_layer = concatenate([hidden_layer_1, hidden_layer_2, hidden_layer_3,hidden_layer_4])

# adiciona a camada de saída com regularização L2
output_layer = Dense(1, activation="relu", kernel_regularizer=regularizers.l2(0.01))(concat_layer)

# cria o modelo
model = Model(inputs=input_layer, outputs=output_layer)

# define a métrica F1-score
f1_score = F1Score(num_classes=1, threshold=0.5)

# cria o otimizador
optimizer = Nadam(learning_rate=0.00001)

# compila o modelo com a métrica F1-score
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[f1_score])


# treina o modelo e salva o modelo com a melhor métrica F1
filepath = "model_f1.h5"
checkpoint = F1ScoreModelCheckpoint(filepath, monitor='val_f1_score', mode='max')
model.fit(X_train_final, y_train_final, epochs=2000, batch_size=127, validation_data=(X_val_final, y_val_final), callbacks=[checkpoint])

# carrega o modelo com a melhor métrica F1
model.load_weights(filepath)

# faz a previsão no conjunto de teste
y_pred = (model.predict(X_val_final) > 0.5)

# avalia o desempenho do modelo
_, f1 = model.evaluate(X_val_final, y_val_final, verbose=0)
print("F1-score:", f1)

